<a href="https://colab.research.google.com/github/ithingv/nlp_study/blob/main/Preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.4 MB/s 
     |████████████████████████████████| 596 kB 54.0 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 61 kB 301 kB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

2. 문장 전처리
- 인코딩

「토크나이저」에는 많은 메소드가 있습니다만, 기억할 필요가 있는 것은 「__call__」뿐입니다. 인스턴스 이름을 메서드처럼 호출합니다

- Input_ids : 토큰 ID의 배열

- token_type_ids : 문장을 결정하기 위한 바이너리 마스크

- attention_mask : 임베딩을 결정하기 위한 바이너리 마스크
    - " Attention_mask "는 모델이 주의를 기울여야 하는 토큰을 결정하는 데 사용됩니다. 1이 주의를 기울여야 하는 토큰, 0이 임베디드를 나타냅니다.

In [3]:
encoded_inputs = tokenizer("I love you")
print(encoded_inputs)

{'input_ids': [101, 146, 1567, 1128, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}



- 디코딩 (토큰 ID 배열 → 문)

- 토크 나이저"는 토큰 ID 배열을 디코딩 할 수도 있습니다.
- 특수 토큰 자동 추가를 사용하지 않으려면 add_special_tokens=False 를 지정합니다. 직접 '스페셜 토큰'을 추가하려는 경우 유용합니다.

In [4]:
tokenizer.decode(encoded_inputs['input_ids'])

'[CLS] I love you [SEP]'

#  복수의 문장의 인코딩
「토크나이저」에 복수의 문장을 리스트로 건네주는 것으로, 정리해 encode 할 수가 있습니다.

- 각 문장을 배치 내의 문장의 최대 길이로 패딩.
- 각 문장을 모델이 허용하는 최대 길이로 잘라냅니다 (해당되는 경우).
- 텐서를 돌려준다.


In [5]:
batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                    "And the very very last one"]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

{'input_ids': [[101, 8667, 146, 112, 182, 170, 1423, 5650, 102], [101, 1262, 1330, 5650, 102], [101, 1262, 1103, 1304, 1304, 1314, 1141, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [6]:
batch = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(batch)

{'input_ids': tensor([[ 101, 8667,  146,  112,  182,  170, 1423, 5650,  102],
        [ 101, 1262, 1330, 5650,  102,    0,    0,    0,    0],
        [ 101, 1262, 1103, 1304, 1304, 1314, 1141,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0]])}


모델과 관련된 최대 길이가 없으면 위 명령은 경고를 발행합니다. 

무시해도 괜찮습니다만, 「verbose=False」를 건네주는 것으로, 경고를 

발행하지 않게 할 수도 있습니다.

# Pair 문장 전처리

모델에 "Pair 문장"을 제공해야 할 수도 있습니다. 

예를 들어, 두 문장이 유사한지 여부를 분류하고 싶은 경우나 문맥과 질문을 하는 질문 응답의 경우 등입니다.

" Token_type_ids "는 어느 부분이 문장 A이고 어느 부분이 문장 B인지 결정하는 데 사용됩니다. " token_type_ids "는 모든 모델에 필요하지 않습니다.

디폴트에서는, 토크나이저의 토큰화는 관련 모델이 기대하는 입력 만을 돌려줍니다. " return_input_ids " 또는 " return_token_type_ids "를 사용하여 이러한 특수 인수를 강제 실행하거나 반환하지 않을 수 있습니다.

In [7]:
encoded_input = tokenizer("How old are you?", "I'm 6 years old")
print(encoded_input)

{'input_ids': [101, 1731, 1385, 1132, 1128, 136, 102, 146, 112, 182, 127, 1201, 1385, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
tokenizer.decode(encoded_input['input_ids'])

"[CLS] How old are you? [SEP] I'm 6 years old [SEP]"

In [9]:
batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                    "And the very very last one"]
batch_of_second_sentences = ["I'm a sentence that goes with the first sentence",
                              "And I should be encoded with the second sentence",
                              "And I go with the very last one"]
encoded_inputs = tokenizer(batch_sentences, batch_of_second_sentences)
print(encoded_inputs)

{'input_ids': [[101, 8667, 146, 112, 182, 170, 1423, 5650, 102, 146, 112, 182, 170, 5650, 1115, 2947, 1114, 1103, 1148, 5650, 102], [101, 1262, 1330, 5650, 102, 1262, 146, 1431, 1129, 12544, 1114, 1103, 1248, 5650, 102], [101, 1262, 1103, 1304, 1304, 1314, 1141, 102, 1262, 146, 1301, 1114, 1103, 1304, 1314, 1141, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [10]:
for ids in encoded_inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] Hello I'm a single sentence [SEP] I'm a sentence that goes with the first sentence [SEP]
[CLS] And another sentence [SEP] And I should be encoded with the second sentence [SEP]
[CLS] And the very very last one [SEP] And I go with the very last one [SEP]


# 패딩 및 잘림 세부 사항


「패딩」과 「자르기」의 상세를 해설합니다. 패딩과 잘라내기 위해 알아야 할 인수는 padding , truncation , max_length 입니다.

- padding
    패딩을 지정합니다. "bool" "문자열"을 지정합니다.

    · false · do_not_pad : 패딩이 수행되지 않습니다.
    · true · longest : 최대 길이로 패딩을 수행합니다.
max_length 가 지정되어 있는 경우는 그 길이, 지정되어 있지 않은 경우는 모델이 허용하는 최대 길이에 패딩 합니다 (max_length=None).

- truncation
    잘림을 지정합니다. "bool" "문자열"을 지정합니다.

    · true · only_first : 최대 길이로 잘립니다.
    · only_second : 문장 쌍의 두 번째 문장을 잘라 냅니다 .
    · longest_first : 토큰별로 잘라 냅니다 . 적절한 길이에 도달할 때까지 쌍 중 가장 긴 문장에서 토큰을 제거합니다.
    ·false · do_not_truncate : 잘림 없음
- max_length
    패딩・자르기의 길이를 지정합니다. 정수, None(모델의 최대 길이)을 지정합니다.

# 사전 토큰화된 문장의 전처리

사전 처리는 사전 토큰화된 입력도 허용합니다. 이는 고유 표현 추출 또는 품사 태그 지정으로 라벨을 계산하여 예측값을 추출하려는 경우에 유효합니다.

【주의】「사전 토큰화」란, 입력이 벌써 토큰화되고 있다고 하는 의미는 아니고 (그렇다면 토큰 라이저에 건네줄 필요는 없습니다), 단어로 분할되고 있을 뿐입니다. 이것은 종종 "BPE"와 같은 서브워드 토큰화 알고리즘의 첫 번째 단계입니다.
사전 토큰화된 입력을 사용하려면, "토크나이저"에 입력을 전달할 때 " is_pretokenized=True "를 지정합니다.

In [11]:
encoded_input = tokenizer()
print(encoded_input)

TypeError: ignored